In [2]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np 
 
 
tokenizer = Tokenizer()
data = open('payas.txt').read()
 
corpus = data.lower().split("\n")
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
 
# create input sequences using list of tokens
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
 
 
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
 
# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
 
label = ku.to_categorical(label, num_classes=total_words)
 
 
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 23, 100)           91700     
                                                                 
 bidirectional (Bidirectiona  (None, 23, 300)          301200    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 23, 300)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               160400    
                                                                 
 dense (Dense)               (None, 458)               46258     
                                                                 
 dense_1 (Dense)             (None, 917)               420903    
                                                        

In [3]:
history = model.fit(predictors, label, epochs=50, verbose=1)


Epoch 1/50
80/80 [==============================] - 15s 118ms/step - loss: 7.1202 - accuracy: 0.0268
Epoch 2/50
80/80 [==============================] - 9s 117ms/step - loss: 6.0622 - accuracy: 0.0399
Epoch 3/50
80/80 [==============================] - 9s 118ms/step - loss: 5.7688 - accuracy: 0.0328
Epoch 4/50
80/80 [==============================] - 10s 131ms/step - loss: 5.6146 - accuracy: 0.0415
Epoch 5/50
80/80 [==============================] - 9s 117ms/step - loss: 5.4820 - accuracy: 0.0450
Epoch 6/50
80/80 [==============================] - 9s 117ms/step - loss: 5.4012 - accuracy: 0.0490
Epoch 7/50
80/80 [==============================] - 9s 118ms/step - loss: 5.3215 - accuracy: 0.0565
Epoch 8/50
80/80 [==============================] - 11s 133ms/step - loss: 5.2510 - accuracy: 0.0608
Epoch 9/50
80/80 [==============================] - 9s 117ms/step - loss: 5.1714 - accuracy: 0.0663
Epoch 10/50
80/80 [==============================] - 9s 118ms/step - loss: 5.1110 - accuracy: 0.0

In [12]:
seed_text = "tierra"
next_words = 30
   
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predict_x=model.predict(token_list, verbose=0) 
    classes_x=np.argmax(predict_x,axis=1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == classes_x:
            output_word = word
            break
    seed_text += " " + output_word
print(seed_text)


tierra no te payas chile son ay brindo dijo un corazón de un moza ay chile son ay brindo por el mujer vino vino son chicha de la ramá brindo por


In [13]:
# Guardar el Modelo
model.save('modelo_payas.h5')

# Recrea exactamente el mismo modelo solo desde el archivo
#new_model = keras.models.load_model('path_to_my_model.h5')

In [14]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
#with open('tokenizer.pickle', 'rb') as handle:
#    tokenizer = pickle.load(handle)

In [15]:
print(max_sequence_len)

24
